In [1]:
#imports
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

#from PathFinder import find_path

In [2]:
#find_path will find the path that data is located in
#storing the data in different locations leads to errors between users
#add your path to the paths array if yours is different

import os

def find_path(verbose=False):
    paths = ["C:/Users/jon.harris/Desktop/school/final/input/", "data/", "../input/"]
    my_path = ""
    for path in paths:
        try:
            os.listdir(path)
            my_path = path
        except:
            filler_exception=0
    if verbose:
        print("path: {}".format(my_path))
        print("all files in directory: {}".format(os.listdir(my_path)))
    return my_path


In [5]:
#read data
PATH = find_path(verbose=True)
PATH += "home-credit-default-risk-preprocessed-data/"

#select files
#files = ['application_train.csv','application_test.csv',]
files = ['preprocessed_train.csv', 'preprocessed_test.csv']
"""         
          'bureau.csv', 
         'bureau_balance.csv', 'credit_card_balance.csv',
         'installments_payments.csv', 'POS_CASH_balance.csv',
         'previous_application.csv']
"""
data_files=[]
data=[]
print("files used: {}".format(files))

#actually read files
for file in files:
    stuff = pd.read_csv(PATH+file,header=0,engine='python')
    #print(stuff)
    data_files.append(file)
    data.append(stuff)

path: ../input/
all files in directory: ['home-credit-default-risk', 'home-credit-default-risk-preprocessed-data']
files used: ['preprocessed_train.csv', 'preprocessed_test.csv']


In [6]:
#select and convert columns
original_train = data[0] #save the original because we'll need some stuff later
original_test = data[1]
app_train = data[0].copy()
app_test = data[1].copy()

#delete ID cols
del app_train['SK_ID_CURR']
del app_test['SK_ID_CURR']

app_train_y = app_train['TARGET']
new_cols_x = app_train.columns.values.tolist()
new_cols_x.remove('TARGET')
app_train_x = app_train[new_cols_x]

print('Training Features shape: ', app_train_x.shape) #235 cols
print('Testing Features shape: ', app_test.shape) #235 cols

Training Features shape:  (307511, 235)
Testing Features shape:  (48744, 235)


In [8]:
#PCA
from sklearn.decomposition import PCA

pca = PCA() #default n_components=235
#pca = PCA(n_components=1) #possibility
#pca = PCA(n_components=2) #possibility
X_train_pca = pca.fit_transform(app_train_x)
X_test_pca = pca.transform(app_test)
#X_train_pca
#X_train_pca.shape #(307511, 235), changes with n_components

In [ ]:
#new dataframe for plotting
#app_train_y.shape
#X_train_lda.shape
#X_train_lda[:,0]
df = pd.DataFrame({'TARGET': app_train_y, 'PCA_c1': X_train_pca[:,0]})
#df = pd.DataFrame({'TARGET': app_train_y, 'PCA_c1': X_train_pca[:,5]}) #all sameish pattern
df.head()

In [ ]:
#plot PCA/Target correlation
#no wonder PCA is so horrible
plt.figure(figsize = (10, 8))

# KDE plot of loans that were repaid on time
sns.kdeplot(df.loc[df['TARGET'] == 0, 'PCA_c1'], label = 'target == 0')

# KDE plot of loans which were not repaid on time
sns.kdeplot(df.loc[df['TARGET'] == 1, 'PCA_c1'], label = 'target == 1')

# Labeling of plot
plt.xlabel('PCA_c1'); plt.ylabel('Density'); plt.title('Distribution of PCA Component 1');

In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_regions(X, y, classifier, resolution=0.02, plot_samples=True):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # plot class samples
    if plot_samples:
        for idx, cl in enumerate(np.unique(y)):
            plt.scatter(x=X[y == cl, 0], 
                        y=X[y == cl, 1],
                        alpha=0.6, 
                        c=cmap(idx),
                        edgecolor='black',
                        marker=markers[idx], 
                        label=cl)

In [13]:
#neural net train
from sklearn.neural_network import MLPRegressor
#mlp = MLPRegressor(hidden_layer_sizes=(300,250,200,150,100,50,40,30,20,10), #possibly too deep lol
mlp = MLPRegressor(hidden_layer_sizes=(300,200,100,50,10), #possibly too deep lol
#                   max_iter=1000)
                   max_iter=100, random_state=20)
mlp.fit(X_train_pca, app_train_y)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(300, 200, 100, 50, 10),
       learning_rate='constant', learning_rate_init=0.001, max_iter=100,
       momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
       power_t=0.5, random_state=20, shuffle=True, solver='adam',
       tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [14]:
# Make predictions
mlp_pred = mlp.predict(X_test_pca)
mlp_pred = mlp_pred.clip(0, 1)

In [15]:
# Submission dataframe
submit = original_test[['SK_ID_CURR']]
submit['TARGET'] = mlp_pred

submit.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,SK_ID_CURR,TARGET
0,100001,0.953197
1,100005,0.005763
2,100013,0.013339
3,100028,0.031067
4,100038,0.014444


In [19]:
# Save the submission to a csv file
overwrite = True
if overwrite:
    submit.to_csv('results/mlp_pca_0.csv', index = False)
    #submit.to_csv('mlp_pca_0.csv', index = False)
    #print(submit.to_csv(index = False))

SK_ID_CURR,TARGET
100001,0.953196558111806
100005,0.005762743341461762
100013,0.013338963162470574
100028,0.031067230839662063
100038,0.014444484699968507
100042,0.03729195660100276
100057,0.013439029645582856
100065,0.03736287469744484
100066,0.008631036166583006
100067,0.00422815858750325
100074,0.031568643871847546
100090,0.01294287220535828
100091,0.04561995595265492
100092,0.036749061835905705
100106,0.007060208938911849
100107,0.01267159145496416
100109,0.02991649568161514
100117,0.017384746000123436
100128,0.012738997703023613
100141,0.017711925854995952
100150,0.0251681122433646
100168,0.009900436658116518
100169,0.012772998455770124
100170,0.023004605411721585
100171,0.03483305406469289
100172,0.04417918896212075
100184,0.04552716405673138
100187,0.013129776346653077
100212,0.0182392480053718
100222,0.049789127346406725
100223,0.005297347407473507
100232,0.011703315680771209
100241,0.004701038733062213
100245,0.023394530649855083
100253,0.02274669548116179
100256,0.02134124182